# 1. 영상을 업로드하면 스쿼트 카운트해줌
* 오른쪽 하지 기준으로 카운트함

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# ======================================
# 💡 각도 계산 함수 (무릎 angle용)
# ======================================
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

# ======================================
# 📁 파일 경로 설정
# ======================================
MODEL_PATH = "/Users/laxdin24/Documents/GitHub/MS_AI_SCHOOL_6/Project All/3차프로젝트/pose_landmarker_heavy.task"    # 다운로드한 .task 모델 경로
VIDEO_PATH = "/Users/laxdin24/Desktop/test_squat.mov"              # 분석할 입력 동영상
OUTPUT_PATH = "/Users/laxdin24/Desktop/out_put_test_squat.mov"             # 저장할 출력 동영상

# ======================================
# 📦 PoseLandmarker 초기화
# ======================================
BaseOptions = python.BaseOptions
VisionRunningMode = vision.RunningMode

options = vision.PoseLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=MODEL_PATH),
    output_segmentation_masks=False,
    running_mode=VisionRunningMode.VIDEO,
    num_poses=1
)
pose_landmarker = vision.PoseLandmarker.create_from_options(options)

# ======================================
# 🎬 비디오 열기 및 저장 설정
# ======================================
cap = cv2.VideoCapture(VIDEO_PATH)

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (width, height))

# ======================================
# 🔁 카운트 로직 변수
# ======================================
counter = 0
stage = None
frame_idx = 0

# ======================================
# 🎯 프레임별 분석 루프
# ======================================
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 입력 이미지 포맷 설정
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # 모델로 포즈 추론
    result = pose_landmarker.detect_for_video(mp_image, frame_idx)
    frame_idx += 1

    try:
        if len(result.pose_landmarks) > 0:
            landmarks = result.pose_landmarks[0]

            # 3개 랜드마크: HIP, KNEE, ANKLE (오른쪽)
            hip   = landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP]
            knee  = landmarks[mp.solutions.pose.PoseLandmark.RIGHT_KNEE]
            ankle = landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ANKLE]

            # (x, y) 좌표만 사용
            hip_point   = [hip.x * width, hip.y * height]
            knee_point  = [knee.x * width, knee.y * height]
            ankle_point = [ankle.x * width, ankle.y * height]

            # 각도 계산
            angle = calculate_angle(hip_point, knee_point, ankle_point)

            # 스쿼트 판단: down → up 변환
            if angle > 160:
                stage = "up"
            elif angle < 100 and stage == "up":
                stage = "down"
                counter += 1
                print(f"스쿼트 {counter}회")

            # 카운트 정보 그리기
            cv2.putText(frame, f'Angle: {int(angle)}', (30, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
            cv2.putText(frame, f'Count: {counter}', (30, 100),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0,255,0), 3)
            cv2.putText(frame, f'Stage: {stage if stage else "-"}', (30, 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,200,255), 2)

    except Exception as e:
        print(f"오류 발생: {e}")

    # 프레임 저장
    out.write(frame)

# 정리
cap.release()
out.release()
pose_landmarker.close()
print(f"\n✅ 최종 스쿼트 횟수: {counter}회")
print(f"📁 결과 영상 저장 완료: {OUTPUT_PATH}")

# 2. 스쿼트 카운트 + 독립적인 좌우하지 피드백

In [ ]:
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# =============================

# 각도 계산 함수
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))

# =============================

# 설정
MODEL_PATH = "/Users/laxdin24/Documents/GitHub/MS_AI_SCHOOL_6/Project All/3차프로젝트/pose_landmarker_heavy.task"    # 다운로드한 .task 모델 경로
VIDEO_PATH = "/Users/laxdin24/Desktop/test_squat_front.mov"              # 분석할 입력 동영상
OUTPUT_PATH = "/Users/laxdin24/Desktop/out_put_test_squat.mov"             # 저장할 출력 동영상
FONT_PATH = "/System/Library/Fonts/Supplemental/AppleGothic.ttf"  # <== 환경에 따라 변경 필요

# =============================

# 모델 초기화
options = vision.PoseLandmarkerOptions(
    base_options=python.BaseOptions(model_asset_path=MODEL_PATH),
    output_segmentation_masks=False,
    running_mode=vision.RunningMode.VIDEO,
    num_poses=1
)
pose_landmarker = vision.PoseLandmarker.create_from_options(options)

cap = cv2.VideoCapture(VIDEO_PATH)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

counter, stage, frame_idx = 0, None, 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB,
                        data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    result = pose_landmarker.detect_for_video(mp_image, frame_idx)
    frame_idx += 1

    try:
        if len(result.pose_landmarks) > 0:
            lm = result.pose_landmarks[0]
            def to_px(lm): return [lm.x * width, lm.y * height]

            # 좌우 관절 포인트
            R = {k: to_px(lm[v]) for k, v in {
                'hip': mp.solutions.pose.PoseLandmark.RIGHT_HIP,
                'knee': mp.solutions.pose.PoseLandmark.RIGHT_KNEE,
                'ankle': mp.solutions.pose.PoseLandmark.RIGHT_ANKLE,
                'shoulder': mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER
            }.items()}

            L = {k: to_px(lm[v]) for k, v in {
                'hip': mp.solutions.pose.PoseLandmark.LEFT_HIP,
                'knee': mp.solutions.pose.PoseLandmark.LEFT_KNEE,
                'ankle': mp.solutions.pose.PoseLandmark.LEFT_ANKLE,
                'shoulder': mp.solutions.pose.PoseLandmark.LEFT_SHOULDER
            }.items()}

            # 각도 계산
            angles = {
                'R-knee': calculate_angle(R['hip'], R['knee'], R['ankle']),
                'R-hip': calculate_angle(R['shoulder'], R['hip'], R['knee']),
                'R-back': calculate_angle(R['shoulder'], R['hip'], R['ankle']),
                'L-knee': calculate_angle(L['hip'], L['knee'], L['ankle']),
                'L-hip': calculate_angle(L['shoulder'], L['hip'], L['knee']),
                'L-back': calculate_angle(L['shoulder'], L['hip'], L['ankle']),
                'R-ankle': calculate_angle(R['knee'], R['ankle'], [R['ankle'][0], R['ankle'][1] + 100]),  # Y축 기준
                'L-ankle': calculate_angle(L['knee'], L['ankle'], [L['ankle'][0], L['ankle'][1] + 100])
            }

            # 카운트 (오른쪽 무릎 기준)
            if angles['R-knee'] > 160:
                stage = "up"
            elif angles['R-knee'] < 140 and stage == "up":
                stage = "down"
                counter += 1

            # 각도 및 카운트 정보 그리기
            cv2.putText(frame, f'count: {counter}', (30, 140),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f'stage: {stage if stage else "-"}', (30, 170),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 255), 2)
            cv2.putText(frame, f"R-knee: {int(angles['R-knee'])}", (30, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            cv2.putText(frame, f"R-hip: {int(angles['R-hip'])}", (30, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            cv2.putText(frame, f"R-ankle: {int(angles['R-ankle'])}", (30, 110),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

            cv2.putText(frame, f"L-knee: {int(angles['L-knee'])}", (250, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
            cv2.putText(frame, f"L-hip: {int(angles['L-hip'])}", (250, 80),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)
            cv2.putText(frame, f"L-ankle: {int(angles['L-ankle'])}", (250, 110),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

    except Exception as e:
        print("오류 발생:", e)

    out.write(frame)

cap.release()
out.release()
pose_landmarker.close()
print(f"\n✅ 스쿼트 {counter}회 완료")
print(f"📁 결과 저장: {OUTPUT_PATH}")

===========================================================
# 3. 🏋️‍♂️ 카운트 + 관절각도 출력 + 무게중심 시각화
===========================================================

In [ ]:
# ==========================
# 📦 1. 라이브러리 로딩 및 설정
# ==========================
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2  # 🔧 변환용

# ==========================
# 📐 2. 유틸 함수 정의
# ==========================

# 각도 계산 함수
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

# Tasks API → Proto 변환 함수 (draw_landmarks용)
def convert_to_proto_landmarks(task_landmarks):
    proto = landmark_pb2.NormalizedLandmarkList()
    for lm in task_landmarks:
        l = proto.landmark.add()
        l.x, l.y, l.z = lm.x, lm.y, lm.z
        l.visibility = getattr(lm, 'visibility', 0)
    return proto

# ==========================
# ⚙️ 3. 파일 경로 설정
# ==========================
MODEL_PATH = "/Users/laxdin24/Documents/GitHub/MS_AI_SCHOOL_6/Project All/3차프로젝트/pose_landmarker_heavy.task"    # 다운로드한 .task 모델 경로
VIDEO_PATH = "/Users/laxdin24/Desktop/test_squat_front.mov"              # 분석할 입력 동영상
OUTPUT_PATH = "/Users/laxdin24/Desktop/out_put_test_squat.mov"             # 저장할 출력 동영상

# ==========================
# 🤖 4. MediaPipe Pose 모델 로딩
# ==========================
options = vision.PoseLandmarkerOptions(
    base_options=python.BaseOptions(model_asset_path=MODEL_PATH),
    running_mode=vision.RunningMode.VIDEO,
    output_segmentation_masks=False,
    num_poses=1
)
pose_landmarker = vision.PoseLandmarker.create_from_options(options)

# MediaPipe 시각화 유틸
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# ==========================
# 🎥 5. 영상 파일 열기
# ==========================
cap = cv2.VideoCapture(VIDEO_PATH)
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)
out    = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

counter, stage, frame_idx = 0, None, 0

# ==========================
# 🔁 6. 프레임 분석 루프
# ==========================
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB,
                        data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    result = pose_landmarker.detect_for_video(mp_image, frame_idx)
    frame_idx += 1

    try:
        if len(result.pose_landmarks) > 0:
            lm = result.pose_landmarks[0]
            def to_px(l): return [l.x * width, l.y * height]

            # ------------------------------------------
            # 📍 6-1. 관절 좌표 추출
            # ------------------------------------------
            R = {k: to_px(lm[v]) for k, v in {
                'hip': mp_pose.PoseLandmark.RIGHT_HIP,
                'knee': mp_pose.PoseLandmark.RIGHT_KNEE,
                'ankle': mp_pose.PoseLandmark.RIGHT_ANKLE,
                'heel': mp_pose.PoseLandmark.RIGHT_HEEL,
                'foot': mp_pose.PoseLandmark.RIGHT_FOOT_INDEX,
                'shoulder': mp_pose.PoseLandmark.RIGHT_SHOULDER
            }.items()}
            L = {k: to_px(lm[v]) for k, v in {
                'hip': mp_pose.PoseLandmark.LEFT_HIP,
                'knee': mp_pose.PoseLandmark.LEFT_KNEE,
                'ankle': mp_pose.PoseLandmark.LEFT_ANKLE,
                'heel': mp_pose.PoseLandmark.LEFT_HEEL,
                'foot': mp_pose.PoseLandmark.LEFT_FOOT_INDEX,
                'shoulder': mp_pose.PoseLandmark.LEFT_SHOULDER
            }.items()}

            # ------------------------------------------
            # 📐 6-2. 각도 계산
            # ------------------------------------------
            angles = {
                'R-knee': calculate_angle(R['hip'], R['knee'], R['ankle']),
                'R-hip':  calculate_angle(R['shoulder'], R['hip'], R['knee']),
                'R-ankle':calculate_angle(R['knee'], R['ankle'], [R['ankle'][0], R['ankle'][1] + 100]),
                'L-knee': calculate_angle(L['hip'], L['knee'], L['ankle']),
                'L-hip':  calculate_angle(L['shoulder'], L['hip'], L['knee']),
                'L-ankle':calculate_angle(L['knee'], L['ankle'], [L['ankle'][0], L['ankle'][1] + 100])
            }

            # ------------------------------------------
            # 🔢 6-3. 스쿼트 카운트 로직
            # ------------------------------------------
            if angles['R-knee'] > 160:
                stage = "up"
            elif angles['R-knee'] < 140 and stage == "up":
                stage = "down"
                counter += 1

            # ------------------------------------------
            # ⚖️ 6-4. 무게중심 계산 + 발바닥 평면 투영
            # ------------------------------------------
            center = np.array([
                (R['ankle'][0] + L['ankle'][0] + R['hip'][0] + L['hip'][0]) / 4,
                (R['ankle'][1] + L['ankle'][1] + R['hip'][1] + L['hip'][1]) / 4
            ], dtype=np.float32).reshape(1, 1, 2)

            foot_plane = np.array([
                to_px(lm[mp_pose.PoseLandmark.LEFT_FOOT_INDEX]),
                to_px(lm[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX]),
                to_px(lm[mp_pose.PoseLandmark.RIGHT_HEEL]),
                to_px(lm[mp_pose.PoseLandmark.LEFT_HEEL])
            ], dtype=np.float32)
            dest_plane = np.array([[0, 0], [300, 0], [300, 300], [0, 300]], dtype=np.float32)
            M = cv2.getPerspectiveTransform(dest_plane, foot_plane)
            com_proj = cv2.perspectiveTransform(np.array([[[150, 150]]], dtype=np.float32), M)
            com_pt = tuple(com_proj[0][0].astype(int))

            # ------------------------------------------
            # 🧍‍♂️ 6-5. 좌우 구분 랜드마크 시각화
            # ------------------------------------------

            # ✅ 좌측 관절 연결 정의
            LEFT_CONNECTIONS = [
                (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW),
                (mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
                (mp_pose.PoseLandmark.LEFT_HIP, mp_pose.PoseLandmark.LEFT_KNEE),
                (mp_pose.PoseLandmark.LEFT_KNEE, mp_pose.PoseLandmark.LEFT_ANKLE)
            ]

            # ✅ 우측 관절 연결 정의
            RIGHT_CONNECTIONS = [
                (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW),
                (mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST),
                (mp_pose.PoseLandmark.RIGHT_HIP, mp_pose.PoseLandmark.RIGHT_KNEE),
                (mp_pose.PoseLandmark.RIGHT_KNEE, mp_pose.PoseLandmark.RIGHT_ANKLE)
            ]

            # 🔁 랜드마크 프로토 타입으로 변환
            proto_landmarks = convert_to_proto_landmarks(lm)
            annotated_frame = frame.copy()

            # 🔵 왼쪽 관절 (하늘색)
            mp_drawing.draw_landmarks(
                image=annotated_frame,
                landmark_list=proto_landmarks,
                connections=LEFT_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=2, circle_radius=3),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=2)
            )

            # 🟠 오른쪽 관절 (주황색)
            mp_drawing.draw_landmarks(
                image=annotated_frame,
                landmark_list=proto_landmarks,
                connections=RIGHT_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 128, 255), thickness=2, circle_radius=3),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 128, 255), thickness=2)
            )

            # 무게중심 표시
            cv2.circle(annotated_frame, com_pt, 6, (0, 0, 255), -1)
            cv2.putText(annotated_frame, "CoM", (com_pt[0] + 10, com_pt[1]),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

            # ------------------------------------------
            # 🖼️ 6-6. 텍스트 시각화
            # ------------------------------------------
            cv2.putText(annotated_frame, f'count: {counter}', (30, 140), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            cv2.putText(annotated_frame, f'stage: {stage if stage else "-"}', (30, 170), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,200,255), 2)

            cv2.putText(annotated_frame, f"R-knee: {int(angles['R-knee'])}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
            cv2.putText(annotated_frame, f"R-hip: {int(angles['R-hip'])}", (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
            cv2.putText(annotated_frame, f"R-ankle: {int(angles['R-ankle'])}", (30, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
            cv2.putText(annotated_frame, f"L-knee: {int(angles['L-knee'])}", (250, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2)
            cv2.putText(annotated_frame, f"L-hip: {int(angles['L-hip'])}", (250, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2)
            cv2.putText(annotated_frame, f"L-ankle: {int(angles['L-ankle'])}", (250, 110), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2)

            frame = annotated_frame

    except Exception as e:
        print("❌ 오류 발생:", e)

    out.write(frame)

# ==========================
# 🔚 7. 마무리 처리
# ==========================
cap.release()
out.release()
pose_landmarker.close()
print(f"📁 저장 완료: {OUTPUT_PATH}")

# 4. 푸쉬업 실수자세 경고하기

In [2]:
# ==============================
# 📦 1. 라이브러리 불러오기
# ==============================
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2
import json

# ==============================
# 📐 2. 각도 계산 함수 정의
# ==============================
def calculate_angle(a, b, c):
    """세 점 기준 각도 (b를 중심으로)"""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

def angle_from_vertical(shoulder, elbow):
    """어깨-팔꿈치 벡터와 수직선(y축) 사이의 각도"""
    vec = elbow - shoulder
    vertical = np.array([0, 1])
    cosine = np.dot(vec, vertical) / (np.linalg.norm(vec) * np.linalg.norm(vertical))
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

# ==============================
# ⚙️ 3. 경로 설정
# ==============================
MODEL_PATH = "/Users/laxdin24/Documents/GitHub/MS_AI_SCHOOL_6/Project All/3차프로젝트/pose_landmarker_heavy.task"
VIDEO_PATH = "/Users/laxdin24/Desktop/Pushup_test1.mov"
OUTPUT_PATH = "/Users/laxdin24/Desktop/output_Pushup_test1.mov"
OUTPUT_JSON = "output_Pushup_test1.json"

# ==============================
# 🎨 4. 시각화 구성
# ==============================
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# ==============================
# 🤖 5. 모델 로딩
# ==============================
options = vision.PoseLandmarkerOptions(
    base_options=python.BaseOptions(model_asset_path=MODEL_PATH),
    running_mode=vision.RunningMode.VIDEO,
    output_segmentation_masks=False,
    num_poses=1
)
pose_landmarker = vision.PoseLandmarker.create_from_options(options)

# ==============================
# 🎥 6. 영상 처리 준비
# ==============================
cap = cv2.VideoCapture(VIDEO_PATH)
width, height = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
log_data = []
frame_idx = 0

# ==============================
# 🔁 7. 프레임 반복 처리
# ==============================
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB,
                        data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    result = pose_landmarker.detect_for_video(mp_image, frame_idx)

    frame_info = {"frame": frame_idx}
    warnings = []

    if result.pose_landmarks:
        lm = result.pose_landmarks[0]
        def to_px(lm): return np.array([int(lm.x * width), int(lm.y * height)])

        # 📍 관절 좌표 추출
        R = {k: to_px(lm[v]) for k, v in {
            'shoulder': mp_pose.PoseLandmark.RIGHT_SHOULDER,
            'hip': mp_pose.PoseLandmark.RIGHT_HIP,
            'knee': mp_pose.PoseLandmark.RIGHT_KNEE,
            'elbow': mp_pose.PoseLandmark.RIGHT_ELBOW,
            'wrist': mp_pose.PoseLandmark.RIGHT_WRIST
        }.items()}
        L = {k: to_px(lm[v]) for k, v in {
            'shoulder': mp_pose.PoseLandmark.LEFT_SHOULDER,
            'elbow': mp_pose.PoseLandmark.LEFT_ELBOW,
            'wrist': mp_pose.PoseLandmark.LEFT_WRIST
        }.items()}
        nose = to_px(lm[mp_pose.PoseLandmark.NOSE])

        # 📐 각도 계산
        body_angle = calculate_angle(R['shoulder'], R['hip'], R['knee'])
        R_spread_angle = angle_from_vertical(R['shoulder'], R['elbow'])
        L_spread_angle = angle_from_vertical(L['shoulder'], L['elbow'])

        # ==============================
        # ⚠️ 자세 오류 감지 블록 (통합버전)
        # ==============================

        # 1️⃣ 허리 정렬 상태 확인 (푸쉬업 시 몸통 일직선 유지 여부)
        if body_angle > 180:
            warnings.append("허리 꺾임")         # ▶️ 허리가 과도하게 내려감
        elif body_angle < 150:
            warnings.append("엉덩이 들림")       # ▶️ 엉덩이가 위로 들림 (V자 형태)

        # 2️⃣ 턱만 내리는 동작 감지 (어깨보다 코가 너무 위일 경우)
        if nose[1] < R['shoulder'][1] - 20:
            warnings.append("턱만 내림")         # ▶️ 몸 전체가 내려오지 않고 턱만 내림

        # 3️⃣ 손목-어깨 수직 정렬 여부 확인 (사선 시점 대응용, x+z축 기준)
        def z(l): return lm[l].z  # z값 접근 함수

        # 오른쪽 손목이 어깨에서 좌우(x) + 앞뒤(z)로 모두 많이 벗어났을 경우
        if (abs(lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].x - lm[mp_pose.PoseLandmark.RIGHT_WRIST].x) > 0.2 and
            abs(z(mp_pose.PoseLandmark.RIGHT_SHOULDER) - z(mp_pose.PoseLandmark.RIGHT_WRIST)) > 0.2):
            warnings.append("오른쪽 손목이 어깨에서 벗어났어요")

        # 왼쪽 손목이 어깨에서 좌우(x) + 앞뒤(z)로 모두 많이 벗어났을 경우
        if (abs(lm[mp_pose.PoseLandmark.LEFT_SHOULDER].x - lm[mp_pose.PoseLandmark.LEFT_WRIST].x) > 0.2 and
            abs(z(mp_pose.PoseLandmark.LEFT_SHOULDER) - z(mp_pose.PoseLandmark.LEFT_WRIST)) > 0.2):
            warnings.append("왼쪽 손목이 어깨에서 벗어났어요")

        # 4️⃣ 손 간격이 너무 좁거나 넓은지 확인 (사선 시점에서도 안정적)
        shoulder_width = abs(lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].x - lm[mp_pose.PoseLandmark.LEFT_SHOULDER].x)
        wrist_width = abs(lm[mp_pose.PoseLandmark.RIGHT_WRIST].x - lm[mp_pose.PoseLandmark.LEFT_WRIST].x)
        wrist_ratio = wrist_width / shoulder_width if shoulder_width > 0 else 1

        if wrist_ratio < 0.4:
            warnings.append("손 간격이 너무 좁아요")   # ▶️ 손이 과도하게 모임
        elif wrist_ratio > 1.6:
            warnings.append("손 간격이 너무 넓어요")   # ▶️ 손이 어깨보다 과도하게 벌어짐

        # 5️⃣ 팔 벌어짐 감지 (어깨–팔꿈치 간 수직 각도 기준)
        if R_spread_angle > 130:
            warnings.append("오른팔 벌어짐")     # ▶️ 오른팔이 몸통에서 과도하게 벌어짐
        if L_spread_angle > 130:
            warnings.append("왼팔 벌어짐")       # ▶️ 왼팔이 몸통에서 과도하게 벌어짐

        # 🧾 프레임 분석 기록
        frame_info.update({
            "body_angle": body_angle,
            "R_spread_angle": R_spread_angle,
            "L_spread_angle": L_spread_angle,
            "warnings": warnings
        })
        log_data.append(frame_info)

        # 🎨 랜드마크 시각화
        proto_landmarks = landmark_pb2.NormalizedLandmarkList()
        for landmark in lm:
            l = proto_landmarks.landmark.add()
            l.x, l.y, l.z = landmark.x, landmark.y, landmark.z

        mp_drawing.draw_landmarks(
            image=frame,
            landmark_list=proto_landmarks,
            connections=mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
        )

    # 🔢 프레임 번호
    cv2.putText(frame, f"Frame: {frame_idx}", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    out.write(frame)
    frame_idx += 1

# ==============================
# 💾 저장 및 종료
# ==============================
cap.release()
out.release()
pose_landmarker.close()

with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(log_data, f, ensure_ascii=False, indent=2)

print(f"✅ 분석 완료!\n📁 영상 저장: {OUTPUT_PATH}\n🧾 JSON 저장: {OUTPUT_JSON}")

I0000 00:00:1747009177.454759 2294921 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1747009177.569022 7346586 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747009177.673081 7346588 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


✅ 분석 완료!
📁 영상 저장: /Users/laxdin24/Desktop/output_Pushup_test1.mov
🧾 JSON 저장: output_Pushup_test1.json


## 푸쉬업을 정측면에서 분석하기

### 뼈대 코드

In [ ]:
# ==============================
# 📦 1. 라이브러리 불러오기
# ==============================
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2
import json

# ==============================
# 📐 2. 각도 계산 함수 정의
# ==============================
def calculate_angle(a, b, c):
    """세 점 기준 각도 (b를 중심으로)"""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

# ==============================
# ⚙️ 3. 경로 설정
# ==============================
MODEL_PATH = "/Users/laxdin24/Documents/GitHub/MS_AI_SCHOOL_6/Project All/3차프로젝트/pose_landmarker_heavy.task"
VIDEO_PATH = "/Users/laxdin24/Desktop/Pushup_test1.mov"
OUTPUT_PATH = "/Users/laxdin24/Desktop/output_Pushup_test1.mov"
OUTPUT_JSON = "output_Pushup_test1.json"

# ==============================
# 🎨 4. 시각화 구성
# ==============================
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# ==============================
# 🤖 5. 모델 로딩
# ==============================
options = vision.PoseLandmarkerOptions(
    base_options=python.BaseOptions(model_asset_path=MODEL_PATH),
    running_mode=vision.RunningMode.VIDEO,
    output_segmentation_masks=False,
    num_poses=1
)
pose_landmarker = vision.PoseLandmarker.create_from_options(options)

# ==============================
# 🎥 6. 영상 처리 준비
# ==============================
cap = cv2.VideoCapture(VIDEO_PATH)
width, height = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
log_data = []
frame_idx = 0

# ==============================
# 🔁 7. 프레임 반복 처리
# ==============================
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB,
                        data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    result = pose_landmarker.detect_for_video(mp_image, frame_idx)

    frame_info = {"frame": frame_idx}
    warnings = []

    if result.pose_landmarks:
        lm = result.pose_landmarks[0]
        def to_px(lm): return np.array([int(lm.x * width), int(lm.y * height)])

        # 📍 관절 좌표 추출
        R = {k: to_px(lm[v]) for k, v in {
            'shoulder': mp_pose.PoseLandmark.RIGHT_SHOULDER,
            'hip': mp_pose.PoseLandmark.RIGHT_HIP,
            'knee': mp_pose.PoseLandmark.RIGHT_KNEE,
            'elbow': mp_pose.PoseLandmark.RIGHT_ELBOW,
            'wrist': mp_pose.PoseLandmark.RIGHT_WRIST
        }.items()}
        L = {k: to_px(lm[v]) for k, v in {
            'shoulder': mp_pose.PoseLandmark.LEFT_SHOULDER,
            'elbow': mp_pose.PoseLandmark.LEFT_ELBOW,
            'wrist': mp_pose.PoseLandmark.LEFT_WRIST,
            'knee': mp_pose.PoseLandmark.LEFT_KNEE,
            'hip': mp_pose.PoseLandmark.LEFT_HIP
        }.items()}
        nose = to_px(lm[mp_pose.PoseLandmark.NOSE])

        # 📐 각도 계산
        
        # ==============================
        # ⚠️ 자세 오류 감지 블록
        # ==============================

        # 🧾 프레임 분석 기록
        frame_info.update({
            "body_angle": body_angle,
            "R_spread_angle": R_spread_angle,
            "L_spread_angle": L_spread_angle,
            "warnings": warnings
        })
        log_data.append(frame_info)

        # 🎨 랜드마크 시각화
        proto_landmarks = landmark_pb2.NormalizedLandmarkList()
        for landmark in lm:
            l = proto_landmarks.landmark.add()
            l.x, l.y, l.z = landmark.x, landmark.y, landmark.z

        mp_drawing.draw_landmarks(
            image=frame,
            landmark_list=proto_landmarks,
            connections=mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
        )

    # 🔢 프레임 번호
    cv2.putText(frame, f"Frame: {frame_idx}", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    out.write(frame)
    frame_idx += 1

# ==============================
# 💾 저장 및 종료
# ==============================
cap.release()
out.release()
pose_landmarker.close()

with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(log_data, f, ensure_ascii=False, indent=2)

print(f"✅ 분석 완료!\n📁 영상 저장: {OUTPUT_PATH}\n🧾 JSON 저장: {OUTPUT_JSON}")

I0000 00:00:1747009177.454759 2294921 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1747009177.569022 7346586 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747009177.673081 7346588 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


✅ 분석 완료!
📁 영상 저장: /Users/laxdin24/Desktop/output_Pushup_test1.mov
🧾 JSON 저장: output_Pushup_test1.json


### 코드 1 카운트, 동작상태, 기본오류

In [3]:
# ==============================
# 📦 1. 라이브러리 불러오기
# ==============================
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2
import json

# ==============================
# 📐 2. 각도 계산 함수 정의
# ==============================
def calculate_angle(a, b, c):
    """세 점 기준 각도 계산 (b 중심)"""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

# ==============================
# ⚙️ 3. 경로 설정
# ==============================
MODEL_PATH = "/Users/laxdin24/Documents/GitHub/MS_AI_SCHOOL_6/Project All/3차프로젝트/pose_landmarker_heavy.task"
VIDEO_PATH = "/Users/laxdin24/Desktop/Pushup_test5.mov"
OUTPUT_PATH = "/Users/laxdin24/Desktop/output_Pushup_test5.mov"
OUTPUT_JSON = "output_Pushup_test5.json"

# ==============================
# 🎨 4. 시각화 및 모델 구성
# ==============================
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

options = vision.PoseLandmarkerOptions(
    base_options=python.BaseOptions(model_asset_path=MODEL_PATH),
    running_mode=vision.RunningMode.VIDEO,
    output_segmentation_masks=False,
    num_poses=1
)
pose_landmarker = vision.PoseLandmarker.create_from_options(options)

# ==============================
# 🎥 5. 영상 준비
# ==============================
cap = cv2.VideoCapture(VIDEO_PATH)
width, height = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
log_data = []
frame_idx = 0

# ==============================
# 🧮 6. 상태 변수 초기화
# ==============================
counter = 0        # 푸쉬업 횟수
stage = None       # 현재 상태: "up" or "down"

# ==============================
# 🔁 7. 프레임 반복 분석
# ==============================
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 📥 MediaPipe용 이미지 변환
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB,
                        data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    result = pose_landmarker.detect_for_video(mp_image, frame_idx)
    
    frame_info = {"frame": frame_idx}
    warnings = []

    if result.pose_landmarks:
        lm = result.pose_landmarks[0]
        def to_px(lm): return np.array([int(lm.x * width), int(lm.y * height)])

        # 📍 주요 관절 좌표 추출
        R = {k: to_px(lm[v]) for k, v in {
            'shoulder': mp_pose.PoseLandmark.RIGHT_SHOULDER,
            'hip': mp_pose.PoseLandmark.RIGHT_HIP,
            'knee': mp_pose.PoseLandmark.RIGHT_KNEE,
            'elbow': mp_pose.PoseLandmark.RIGHT_ELBOW,
            'wrist': mp_pose.PoseLandmark.RIGHT_WRIST
        }.items()}
        nose = to_px(lm[mp_pose.PoseLandmark.NOSE])

        # 📐 각도 계산
        body_angle = calculate_angle(R['shoulder'], R['hip'], R['knee'])
        elbow_angle = calculate_angle(R['shoulder'], R['elbow'], R['wrist'])

        # 🔢 푸쉬업 카운트 로직 (elbow 기준)
        if elbow_angle > 160:
            stage = "up"
        elif elbow_angle < 90 and stage == "up":
            stage = "down"
            counter += 1

        # ⚠️ 오류 감지
        if body_angle > 180:
            warnings.append("허리 꺾임")
        elif body_angle < 150:
            warnings.append("엉덩이 들림")
        if nose[1] < R['shoulder'][1] - 20:
            warnings.append("턱만 내림")

        # 🧾 로그 저장
        frame_info.update({
            "elbow_angle": elbow_angle,
            "body_angle": body_angle,
            "stage": stage,
            "count": counter,
            "warnings": warnings
        })
        log_data.append(frame_info)

        # 🎨 랜드마크 시각화
        proto_landmarks = landmark_pb2.NormalizedLandmarkList()
        for landmark in lm:
            l = proto_landmarks.landmark.add()
            l.x, l.y, l.z = landmark.x, landmark.y, landmark.z

        mp_drawing.draw_landmarks(
            image=frame,
            landmark_list=proto_landmarks,
            connections=mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
        )

    # 🔢 시각정보 출력
    cv2.putText(frame, f"Frame: {frame_idx}", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.putText(frame, f"Count: {counter}", (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)
    if stage:
        cv2.putText(frame, f"Stage: {stage}", (30, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,200,255), 2)

    out.write(frame)
    frame_idx += 1

# ==============================
# 💾 저장 및 종료 처리
# ==============================
cap.release()
out.release()
pose_landmarker.close()

with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(log_data, f, ensure_ascii=False, indent=2)

print(f"✅ 분석 완료!\n📁 영상 저장: {OUTPUT_PATH}\n🧾 JSON 저장: {OUTPUT_JSON}")

I0000 00:00:1747010306.278433 2294921 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1747010306.357674 7380037 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747010306.428867 7380043 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


✅ 분석 완료!
📁 영상 저장: /Users/laxdin24/Desktop/output_Pushup_test5.mov
🧾 JSON 저장: output_Pushup_test5.json


### 코드2 기본오류

In [ ]:
# ==============================
# 📦 1. 라이브러리 불러오기
# ==============================
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2
import json

# ==============================
# 📐 2. 유틸 함수 정의
# ==============================

def calculate_angle(a, b, c):
    """세 점을 기준으로 중심 b에서의 각도 계산"""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

# ==============================
# ⚙️ 3. 경로 설정
# ==============================
MODEL_PATH = "/Users/laxdin24/Documents/GitHub/MS_AI_SCHOOL_6/Project All/3차프로젝트/pose_landmarker_heavy.task"
VIDEO_PATH = "/Users/laxdin24/Desktop/Pushup_test5.mov"
OUTPUT_PATH = "/Users/laxdin24/Desktop/output_Pushup_test5.mov"
OUTPUT_JSON = "output_Pushup_test5.json"

# ==============================
# 🎨 4. MediaPipe 설정
# ==============================
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

options = vision.PoseLandmarkerOptions(
    base_options=python.BaseOptions(model_asset_path=MODEL_PATH),
    running_mode=vision.RunningMode.VIDEO,
    output_segmentation_masks=False,
    num_poses=1
)
pose_landmarker = vision.PoseLandmarker.create_from_options(options)

# ==============================
# 🎥 5. 영상 입출력 설정
# ==============================
cap = cv2.VideoCapture(VIDEO_PATH)
width, height = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

frame_idx = 0
log_data = []

# ==============================
# 🔁 6. 프레임 분석 루프
# ==============================
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 🎯 MediaPipe용 이미지 준비
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB,
                        data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    result = pose_landmarker.detect_for_video(mp_image, frame_idx)

    frame_info = {"frame": frame_idx}
    warnings = []

    if result.pose_landmarks:
        lm = result.pose_landmarks[0]
        def to_np(l): return np.array([lm[l].x * width, lm[l].y * height])

        # 🧠 좌우 어깨 visibility 비교 → 잘 보이는 쪽 선택
        r_vis = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].visibility
        l_vis = lm[mp_pose.PoseLandmark.LEFT_SHOULDER].visibility
        side = 'RIGHT' if r_vis > l_vis else 'LEFT'

        # 📍 선택된 방향 기준 관절 추출
        if side == 'RIGHT':
            SHOULDER = to_np(mp_pose.PoseLandmark.RIGHT_SHOULDER)
            ELBOW    = to_np(mp_pose.PoseLandmark.RIGHT_ELBOW)
            WRIST    = to_np(mp_pose.PoseLandmark.RIGHT_WRIST)
            HIP      = to_np(mp_pose.PoseLandmark.RIGHT_HIP)
            KNEE     = to_np(mp_pose.PoseLandmark.RIGHT_KNEE)
            SHOULDER_X = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].x
            WRIST_X    = lm[mp_pose.PoseLandmark.RIGHT_WRIST].x
        else:
            SHOULDER = to_np(mp_pose.PoseLandmark.LEFT_SHOULDER)
            ELBOW    = to_np(mp_pose.PoseLandmark.LEFT_ELBOW)
            WRIST    = to_np(mp_pose.PoseLandmark.LEFT_WRIST)
            HIP      = to_np(mp_pose.PoseLandmark.LEFT_HIP)
            KNEE     = to_np(mp_pose.PoseLandmark.LEFT_KNEE)
            SHOULDER_X = lm[mp_pose.PoseLandmark.LEFT_SHOULDER].x
            WRIST_X    = lm[mp_pose.PoseLandmark.LEFT_WRIST].x

        NOSE = to_np(mp_pose.PoseLandmark.NOSE)

        # 📐 각도 계산
        body_angle = calculate_angle(SHOULDER, HIP, KNEE)
        elbow_angle = calculate_angle(SHOULDER, ELBOW, WRIST)

        # ⚠️ 오류 감지 (정측면용)
        if body_angle > 180:
            warnings.append("허리 꺾임")
        elif body_angle < 150:
            warnings.append("엉덩이 들림")
        if NOSE[1] < SHOULDER[1] - 20:
            warnings.append("턱만 내림")

        x_diff = SHOULDER_X - WRIST_X # 부호 유지!

        if x_diff < -0.2:
            warnings.append(f"손목이 위쪽(머리 방향)에 치우쳐 있어요. 팔꿈치 각도는 {elbow_angle:.1f}° 입니다.")
        elif x_diff > 0.2:
            warnings.append(f"손목이 아래쪽(엉덩이 방향)에 치우쳐 있어요. 팔꿈치 각도는 {elbow_angle:.1f}° 입니다.")

        # 📝 분석 결과 기록
        frame_info.update({
            "side_used": side,
            "body_angle": body_angle,
            "elbow_angle": elbow_angle,
            "x_diff": x_diff,
            "warnings": warnings
        })
        log_data.append(frame_info)

        # 🎨 랜드마크 시각화
        proto_landmarks = landmark_pb2.NormalizedLandmarkList()
        for landmark in lm:
            l = proto_landmarks.landmark.add()
            l.x, l.y, l.z = landmark.x, landmark.y, landmark.z

        mp_drawing.draw_landmarks(
            image=frame,
            landmark_list=proto_landmarks,
            connections=mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
        )

    # 📺 프레임 시각 정보 출력
    cv2.putText(frame, f"Frame: {frame_idx}", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    out.write(frame)
    frame_idx += 1

# ==============================
# 💾 7. 저장 및 종료 처리
# ==============================
cap.release()
out.release()
pose_landmarker.close()

with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(log_data, f, ensure_ascii=False, indent=2)

print(f"✅ 분석 완료!\n📁 영상 저장: {OUTPUT_PATH}\n🧾 JSON 저장: {OUTPUT_JSON}")

I0000 00:00:1747015524.971112 7518703 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1747015525.048159 7533604 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747015525.125655 7533598 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


✅ 분석 완료!
📁 영상 저장: /Users/laxdin24/Desktop/output_Pushup_test5.mov
🧾 JSON 저장: output_Pushup_test5.json


### 코드 3 기본오류 궤적 그리기

In [4]:
# ==============================
# 📦 1. 라이브러리 불러오기
# ==============================
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2
import json

# ==============================
# 📐 2. 유틸 함수 정의
# ==============================
def calculate_angle(a, b, c):
    """세 점 기준으로 중심 b에서의 각도 계산"""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

# ==============================
# ⚙️ 3. 경로 설정
# ==============================
MODEL_PATH = "/Users/laxdin24/Documents/GitHub/MS_AI_SCHOOL_6/Project All/3차프로젝트/pose_landmarker_heavy.task"
VIDEO_PATH = "/Users/laxdin24/Desktop/Pushup_test4.mov"
OUTPUT_PATH = "/Users/laxdin24/Desktop/output_Pushup_test4.mov"
OUTPUT_JSON = "output_Pushup_test4.json"

# ==============================
# 🎨 4. MediaPipe 구성
# ==============================
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

options = vision.PoseLandmarkerOptions(
    base_options=python.BaseOptions(model_asset_path=MODEL_PATH),
    running_mode=vision.RunningMode.VIDEO,
    num_poses=1,
    output_segmentation_masks=False
)
pose_landmarker = vision.PoseLandmarker.create_from_options(options)

# ==============================
# 🎥 5. 영상 설정
# ==============================
cap = cv2.VideoCapture(VIDEO_PATH)
width, height = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# 💾 로그 및 프레임 설정
frame_idx = 0
log_data = []
trajectory = {"RIGHT": [], "LEFT": []}  # 어깨 궤적 저장용
trajectory_len = int(fps * 2)           # 2초 유지 (예: 30fps → 60프레임)

# ==============================
# 🔁 6. 프레임 처리 루프
# ==============================
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # ✅ MediaPipe 입력 처리
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB,
                        data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    result = pose_landmarker.detect_for_video(mp_image, frame_idx)
    frame_info = {"frame": frame_idx}
    warnings = []

    if result.pose_landmarks:
        lm = result.pose_landmarks[0]
        def to_np(l): return np.array([lm[l].x * width, lm[l].y * height])

        # 🔍 어깨 visibility 기반 우선 사용 방향 결정
        r_vis = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].visibility
        l_vis = lm[mp_pose.PoseLandmark.LEFT_SHOULDER].visibility
        side = 'RIGHT' if r_vis > l_vis else 'LEFT'

        # 📍 관절 좌표 추출
        if side == 'RIGHT':
            SHOULDER = to_np(mp_pose.PoseLandmark.RIGHT_SHOULDER)
            ELBOW    = to_np(mp_pose.PoseLandmark.RIGHT_ELBOW)
            WRIST    = to_np(mp_pose.PoseLandmark.RIGHT_WRIST)
            HIP      = to_np(mp_pose.PoseLandmark.RIGHT_HIP)
            KNEE     = to_np(mp_pose.PoseLandmark.RIGHT_KNEE)
            SHOULDER_X = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].x
            WRIST_X    = lm[mp_pose.PoseLandmark.RIGHT_WRIST].x
        else:
            SHOULDER = to_np(mp_pose.PoseLandmark.LEFT_SHOULDER)
            ELBOW    = to_np(mp_pose.PoseLandmark.LEFT_ELBOW)
            WRIST    = to_np(mp_pose.PoseLandmark.LEFT_WRIST)
            HIP      = to_np(mp_pose.PoseLandmark.LEFT_HIP)
            KNEE     = to_np(mp_pose.PoseLandmark.LEFT_KNEE)
            SHOULDER_X = lm[mp_pose.PoseLandmark.LEFT_SHOULDER].x
            WRIST_X    = lm[mp_pose.PoseLandmark.LEFT_WRIST].x

        NOSE = to_np(mp_pose.PoseLandmark.NOSE)

        # 📐 각도 계산
        body_angle = calculate_angle(SHOULDER, HIP, KNEE)
        elbow_angle = calculate_angle(SHOULDER, ELBOW, WRIST)

        # ⚠️ 자세 오류 감지
        if body_angle > 180:
            warnings.append("허리 꺾임")
        elif body_angle < 150:
            warnings.append("엉덩이 들림")
        if NOSE[1] < SHOULDER[1] - 20:
            warnings.append("턱만 내림")

        # ✅ 손목의 좌우 위치 평가 (x 좌표 차이)
        x_diff = SHOULDER_X - WRIST_X
        if x_diff < -0.2:
            warnings.append(f"손목이 머리쪽으로 치우쳐 있어요 ({elbow_angle:.1f}°)")
        elif x_diff > 0.2:
            warnings.append(f"손목이 엉덩이쪽으로 치우쳐 있어요 ({elbow_angle:.1f}°)")

        # 🧾 JSON 로그 저장
        frame_info.update({
            "side_used": side,
            "body_angle": body_angle,
            "elbow_angle": elbow_angle,
            "x_diff": x_diff,
            "warnings": warnings
        })
        log_data.append(frame_info)

        # ==============================
        # ✏️ 어깨 궤적 기록 및 시각화 (2초 유지)
        # ==============================
        pts = trajectory[side]
        pts.append((int(SHOULDER[0]), int(SHOULDER[1])))
        if len(pts) > trajectory_len:
            pts.pop(0)

        for i in range(1, len(pts)):
            cv2.line(frame, pts[i - 1], pts[i], (0, 255, 255), 2)  # 노란색 선

        # ==============================
        # 🖼️ 랜드마크 시각화
        # ==============================
        proto_landmarks = landmark_pb2.NormalizedLandmarkList()
        for landmark in lm:
            l = proto_landmarks.landmark.add()
            l.x, l.y, l.z = landmark.x, landmark.y, landmark.z

        mp_drawing.draw_landmarks(
            image=frame,
            landmark_list=proto_landmarks,
            connections=mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
        )

    # 📺 프레임 텍스트 시각화
    cv2.putText(frame, f"Frame: {frame_idx}", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    out.write(frame)
    frame_idx += 1

# ==============================
# 💾 7. 종료 처리
# ==============================
cap.release()
out.release()
pose_landmarker.close()

with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(log_data, f, ensure_ascii=False, indent=2)

print(f"✅ 분석 완료!\n📁 영상 저장: {OUTPUT_PATH}\n🧾 JSON 저장: {OUTPUT_JSON}")

I0000 00:00:1747016290.723563 7518703 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1747016290.787338 7556128 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747016290.838479 7556131 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


✅ 분석 완료!
📁 영상 저장: /Users/laxdin24/Desktop/output_Pushup_test4.mov
🧾 JSON 저장: output_Pushup_test4.json


### 코드 4 기본오류, 궤적그리기, 동작상태 및 카운트

In [ ]:
# ==============================
# 📦 1. 라이브러리 불러오기
# ==============================
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe.framework.formats import landmark_pb2
import json

# ==============================
# 📐 2. 각도 계산 함수 정의
# ==============================
def calculate_angle(a, b, c):
    """세 점 기준으로 중심 b에서의 각도 계산"""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

# ==============================
# ⚙️ 3. 경로 설정
# ==============================
MODEL_PATH = "/Users/laxdin24/Documents/GitHub/MS_AI_SCHOOL_6/Project All/3차프로젝트/pose_landmarker_heavy.task"
VIDEO_PATH = "/Users/laxdin24/Desktop/Nomal_test1.mov"
OUTPUT_PATH = "/Users/laxdin24/Desktop/output_Nomal_test1.mov"
OUTPUT_JSON = "output_Nomal_test1.json"

# ==============================
# 🎨 4. MediaPipe 구성
# ==============================
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

options = vision.PoseLandmarkerOptions(
    base_options=python.BaseOptions(model_asset_path=MODEL_PATH),
    running_mode=vision.RunningMode.VIDEO,
    output_segmentation_masks=False,
    num_poses=1
)
pose_landmarker = vision.PoseLandmarker.create_from_options(options)

# ==============================
# 🎥 5. 영상 설정
# ==============================
cap = cv2.VideoCapture(VIDEO_PATH)
width, height = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
frame_idx = 0
log_data = []

# 푸쉬업상태 변수 초기화
state = "Unknown"
prev_elbow_angle = None
counter = 0

# 어깨이동경로 로그 및 프레임 설정
trajectory = {"RIGHT": [], "LEFT": []}  # 어깨 궤적 저장용
trajectory_len = int(fps * 2)           # 2초 유지 (예: 30fps → 60프레임)

# ==============================
# 🔁 6. 프레임 처리 루프
# ==============================
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # ✅ MediaPipe 입력 처리
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB,
                        data=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    result = pose_landmarker.detect_for_video(mp_image, frame_idx)
    frame_info = {"frame": frame_idx}
    warnings = []

    if result.pose_landmarks:
        lm = result.pose_landmarks[0]
        def to_np(l): return np.array([lm[l].x * width, lm[l].y * height])

        # 어깨 visibility 비교 → 더 잘 보이는 쪽 사용
        r_vis = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].visibility
        l_vis = lm[mp_pose.PoseLandmark.LEFT_SHOULDER].visibility
        side = 'RIGHT' if r_vis > l_vis else 'LEFT'

        # 관절 좌표 추출 (선택된 방향)
        if side == 'RIGHT':
            SHOULDER = to_np(mp_pose.PoseLandmark.RIGHT_SHOULDER)
            ELBOW    = to_np(mp_pose.PoseLandmark.RIGHT_ELBOW)
            WRIST    = to_np(mp_pose.PoseLandmark.RIGHT_WRIST)
            HIP      = to_np(mp_pose.PoseLandmark.RIGHT_HIP)
            KNEE     = to_np(mp_pose.PoseLandmark.RIGHT_KNEE)
            SHOULDER_X = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].x
            WRIST_X    = lm[mp_pose.PoseLandmark.RIGHT_WRIST].x
        else:
            SHOULDER = to_np(mp_pose.PoseLandmark.LEFT_SHOULDER)
            ELBOW    = to_np(mp_pose.PoseLandmark.LEFT_ELBOW)
            WRIST    = to_np(mp_pose.PoseLandmark.LEFT_WRIST)
            HIP      = to_np(mp_pose.PoseLandmark.LEFT_HIP)
            KNEE     = to_np(mp_pose.PoseLandmark.LEFT_KNEE)
            SHOULDER_X = lm[mp_pose.PoseLandmark.LEFT_SHOULDER].x
            WRIST_X    = lm[mp_pose.PoseLandmark.LEFT_WRIST].x

        NOSE = to_np(mp_pose.PoseLandmark.NOSE)

        # ==============================
        # 📐 각도 계산
        # ==============================
        body_angle = calculate_angle(SHOULDER, HIP, KNEE)
        elbow_angle = calculate_angle(SHOULDER, ELBOW, WRIST)

        # ==============================
        # ⚠️ 오류 감지
        # ==============================
        if body_angle > 175:
            warnings.append("허리 꺾임")
        elif body_angle < 150:
            warnings.append("엉덩이 들림")
        if NOSE[1] < SHOULDER[1] - 20:
            warnings.append("턱만 내림")

        # 📏 손목의 위치 평가 (x축 방향 기준)
        x_diff = SHOULDER_X - WRIST_X
        if x_diff < -0.2:
            warnings.append(f"손목이 머리쪽으로 치우쳐 있어요 (팔꿈치각도{elbow_angle:.1f}°)")
        elif x_diff > 0.2:
            warnings.append(f"손목이 엉덩이쪽으로 치우쳐 있어요 (팔꿈치각도{elbow_angle:.1f}°)")

        # ==============================
        # 🔁 상태 전이 기반 카운트 로직
        # ==============================
        if prev_elbow_angle is not None:
            # delta = abs(elbow_angle - prev_elbow_angle)

            # if delta > 3:
            #     current_state = "Moving"
            # else:
                if elbow_angle > 140:
                    current_state = "Up"
                elif elbow_angle < 140 and elbow_angle > 100:
                    current_state = "Mid"
                elif elbow_angle < 100:
                    current_state = "Down"
                # else:
                #     current_state = "Moving"
        # else:
        #     current_state = "Unknown"

        # 카운트 조건: Mid → Up 전이 감지
        if state == "Mid" and current_state == "Up":
            counter += 1

        state = current_state
        prev_elbow_angle = elbow_angle

        # 🧾 분석 결과 저장
        frame_info.update({
            "side_used": side,
            "body_angle": body_angle,
            "elbow_angle": elbow_angle,
            "x_diff": x_diff,
            "state": current_state,
            "count": counter,
            "warnings": warnings
        })
        log_data.append(frame_info)

        # ==============================
        # ✏️ 어깨 궤적 기록 및 시각화 (2초 유지)
        # ==============================
        pts = trajectory[side]
        pts.append((int(SHOULDER[0]), int(SHOULDER[1])))
        if len(pts) > trajectory_len:
            pts.pop(0)

        for i in range(1, len(pts)):
            cv2.line(frame, pts[i - 1], pts[i], (0, 255, 255), 2)  # 노란색 선

        # ==============================
        # 🎨 랜드마크 시각화
        # ==============================
        proto_landmarks = landmark_pb2.NormalizedLandmarkList()
        for landmark in lm:
            l = proto_landmarks.landmark.add()
            l.x, l.y, l.z = landmark.x, landmark.y, landmark.z

        mp_drawing.draw_landmarks(
            image=frame,
            landmark_list=proto_landmarks,
            connections=mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
            connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
        )

    # ==============================
    # 🖼️ 프레임 텍스트 출력
    # ==============================
    cv2.putText(frame, f"Frame: {frame_idx}", (30, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.putText(frame, f"Count: {counter}", (30, 80),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)
    if state:
        cv2.putText(frame, f"State: {state}", (30, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,200,255), 2)

    out.write(frame)
    frame_idx += 1

# ==============================
# 💾 저장 및 종료 처리
# ==============================
cap.release()
out.release()
pose_landmarker.close()

with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(log_data, f, ensure_ascii=False, indent=2)

print(f"✅ 분석 완료!\n📁 영상 저장: {OUTPUT_PATH}\n🧾 JSON 저장: {OUTPUT_JSON}")

I0000 00:00:1747030485.124667 7594167 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1747030485.191670 7941678 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747030485.252284 7941680 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


✅ 분석 완료!
📁 영상 저장: /Users/laxdin24/Desktop/output_Nomal_test1.mov
🧾 JSON 저장: output_Nomal_test1.json


### Solutions API 사용

In [ ]:
# ==============================
# 📦 1. 라이브러리 불러오기
# ==============================
import cv2
import mediapipe as mp
import numpy as np
import json

# ==============================
# 📐 2. 각도 계산 함수 정의
# ==============================
def calculate_angle(a, b, c):
    """세 점 기준으로 중심 b에서의 각도 계산"""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

# ==============================
# ⚙️ 3. 기본 설정
# ==============================
VIDEO_PATH = "/Users/laxdin24/Desktop/Nomal_test1.mov"
OUTPUT_PATH = "/Users/laxdin24/Desktop/output_Nomal_test1.mov"
OUTPUT_JSON = "output_Nomal_test1.json"

# MediaPipe 구성
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# ==============================
# 🎥 4. 영상 입출력 준비
# ==============================
cap = cv2.VideoCapture(VIDEO_PATH)
width, height = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# 초기화 변수
frame_idx = 0
log_data = []
state = "Unknown"
prev_elbow_angle = None
counter = 0

# ==============================
# 🔁 5. 프레임 처리 루프 시작
# ==============================
with mp_pose.Pose(
    static_image_mode=False,
    model_complexity=2,                # Heavy 모델
    enable_segmentation=False,
    smooth_landmarks=True,
    min_detection_confidence=0.2,
    min_tracking_confidence=0.2
) as pose:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # ✅ BGR → RGB 변환 후 MediaPipe 처리
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        # ✅ 다시 BGR 변환 및 쓰기 가능 상태 복구
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        frame_info = {"frame": frame_idx}
        warnings = []

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark

            # 관절 좌표 → numpy 변환 함수
            def to_np(idx): return np.array([lm[idx].x * width, lm[idx].y * height])
            def get_x(idx): return lm[idx].x

            # 좌우 어깨 visibility로 잘 보이는 쪽 자동 선택
            r_vis = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].visibility
            l_vis = lm[mp_pose.PoseLandmark.LEFT_SHOULDER].visibility
            side = 'RIGHT' if r_vis > l_vis else 'LEFT'

            # 사용 좌표 선택
            if side == "RIGHT":
                SHOULDER = to_np(mp_pose.PoseLandmark.RIGHT_SHOULDER)
                ELBOW    = to_np(mp_pose.PoseLandmark.RIGHT_ELBOW)
                WRIST    = to_np(mp_pose.PoseLandmark.RIGHT_WRIST)
                HIP      = to_np(mp_pose.PoseLandmark.RIGHT_HIP)
                KNEE     = to_np(mp_pose.PoseLandmark.RIGHT_KNEE)
                SHOULDER_X = get_x(mp_pose.PoseLandmark.RIGHT_SHOULDER)
                WRIST_X    = get_x(mp_pose.PoseLandmark.RIGHT_WRIST)
            else:
                SHOULDER = to_np(mp_pose.PoseLandmark.LEFT_SHOULDER)
                ELBOW    = to_np(mp_pose.PoseLandmark.LEFT_ELBOW)
                WRIST    = to_np(mp_pose.PoseLandmark.LEFT_WRIST)
                HIP      = to_np(mp_pose.PoseLandmark.LEFT_HIP)
                KNEE     = to_np(mp_pose.PoseLandmark.LEFT_KNEE)
                SHOULDER_X = get_x(mp_pose.PoseLandmark.LEFT_SHOULDER)
                WRIST_X    = get_x(mp_pose.PoseLandmark.LEFT_WRIST)

            NOSE = to_np(mp_pose.PoseLandmark.NOSE)

            # =============================
            # 📐 각도 계산
            # =============================
            body_angle = calculate_angle(SHOULDER, HIP, KNEE)
            elbow_angle = calculate_angle(SHOULDER, ELBOW, WRIST)

            # =============================
            # ⚠️ 오류 감지
            # =============================
            if body_angle > 175:
                warnings.append("허리 꺾임")
            elif body_angle < 150:
                warnings.append("엉덩이 들림")
            if NOSE[1] < SHOULDER[1] - 20:
                warnings.append("턱만 내림")

            # 📏 손목 위치가 너무 위/아래로 벗어난 경우
            x_diff = SHOULDER_X - WRIST_X
            if x_diff < -0.2:
                warnings.append(f"손목이 머리 방향으로 치우침 (팔꿈치각도: {elbow_angle:.1f}°)")
            elif x_diff > 0.2:
                warnings.append(f"손목이 엉덩이 방향으로 치우침 (팔꿈치각도: {elbow_angle:.1f}°)")

            # =============================
            # 🔁 상태 전이 및 카운트 로직
            # =============================
            if prev_elbow_angle is not None:
                if elbow_angle > 140:
                    current_state = "Up"
                elif elbow_angle < 140 and elbow_angle > 100:
                    current_state = "Mid"
                elif elbow_angle < 100:
                    current_state = "Down"
            else:
                current_state = "Unknown"

            # 카운트 조건: Mid → Up 전이
            if state == "Mid" and current_state == "Up":
                counter += 1

            state = current_state
            prev_elbow_angle = elbow_angle

            # =============================
            # 🧾 프레임 정보 저장
            # =============================
            frame_info.update({
                "side_used": side,
                "body_angle": body_angle,
                "elbow_angle": elbow_angle,
                "x_diff": x_diff,
                "state": current_state,
                "count": counter,
                "warnings": warnings
            })
            log_data.append(frame_info)

            # =============================
            # 🎨 랜드마크 시각화
            # =============================
            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=results.pose_landmarks,
                connections=mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_styles.get_default_pose_landmarks_style(),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255), thickness=2)
            )

        # =============================
        # 🖼️ 화면 정보 시각화
        # =============================
        cv2.putText(image, f"Frame: {frame_idx}", (30, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        cv2.putText(image, f"Count: {counter}", (30, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)
        if state:
            cv2.putText(image, f"State: {state}", (30, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0,200,255), 2)

        # 💾 영상 출력
        out.write(image)
        frame_idx += 1

# =============================
# 💾 저장 및 종료a
# =============================
cap.release()
out.release()
with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(log_data, f, ensure_ascii=False, indent=2)

print(f"✅ 분석 완료!\n📁 영상 저장: {OUTPUT_PATH}\n🧾 JSON 저장: {OUTPUT_JSON}")

I0000 00:00:1747032034.839569 7594167 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1747032034.996298 7987251 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747032035.105336 7987251 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


✅ 분석 완료!
📁 영상 저장: /Users/laxdin24/Desktop/output_Nomal_test1.mov
🧾 JSON 저장: output_Nomal_test1.json


### 푸쉬업 어깨추척, 정보저장

In [14]:
# 1. 라이브러리 불러오기
import cv2
import mediapipe as mp
import numpy as np
import json

# 2. 각도 계산 함수 정의
def calculate_angle(a, b, c):
    """
    중심점 b 기준으로 a-b-c 사이 각도 계산
    팔꿈치와 다른 관절의 각도 계산에 사용됨
    """
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

# 3. 기본 설정
VIDEO_PATH = "/Users/laxdin24/Desktop/wrong_test1.mov"
OUTPUT_PATH = "/Users/laxdin24/Desktop/output_wrong_test1.mov"
OUTPUT_JSON = "output_wrong_test1.json"

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# 4. 영상 입출력 준비
cap = cv2.VideoCapture(VIDEO_PATH)
width, height = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# 기능을 구현하기위한 각종 초기 변수 선언
frame_idx = 0 # 프레임 인덱스 초기화
log_data = []   # JSON 로그 저장용
state = "Unknown"   # 푸쉬업 상태 초기화
prev_elbow_angle = None # 이전 팔꿈치 각도 초기화
counter = 0 # 카운트 초기화

# 어깨 이동 경로 기록용: 2초치 저장 (예: 30fps → 60프레임)
trajectory = {"RIGHT": [], "LEFT": []}
trajectory_len = int(fps * 1)

# 5. 프레임 처리 루프
with mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    smooth_landmarks=True,
    min_detection_confidence=0.9,
    min_tracking_confidence=0.5
) as pose:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # BGR → RGB 변환 및 MediaPipe 처리
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        frame_info = {"frame": frame_idx}

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark

            # 포인트 변환 함수
            def to_np(idx): return np.array([lm[idx].x * width, lm[idx].y * height])
            def get_x(idx): return lm[idx].x

            # 잘 보이는 쪽 어깨 선택
            r_vis = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].visibility
            l_vis = lm[mp_pose.PoseLandmark.LEFT_SHOULDER].visibility
            side = 'RIGHT' if r_vis > l_vis else 'LEFT'

            # 사용 방향에 따른 관절 추출
            if side == "RIGHT":
                SHOULDER = to_np(mp_pose.PoseLandmark.RIGHT_SHOULDER)
                ELBOW    = to_np(mp_pose.PoseLandmark.RIGHT_ELBOW)
                WRIST    = to_np(mp_pose.PoseLandmark.RIGHT_WRIST)
            else:
                SHOULDER = to_np(mp_pose.PoseLandmark.LEFT_SHOULDER)
                ELBOW    = to_np(mp_pose.PoseLandmark.LEFT_ELBOW)
                WRIST    = to_np(mp_pose.PoseLandmark.LEFT_WRIST)

            # 6. 팔꿈치 각도 계산
            elbow_angle = calculate_angle(SHOULDER, ELBOW, WRIST)

            # 7. 상태 판단 및 카운트 전이 감지
            if prev_elbow_angle is not None:
                if elbow_angle > 140:
                    current_state = "Up"
                elif 100 < elbow_angle <= 140:
                    current_state = "Mid"
                else:
                    current_state = "Down"
            else:
                current_state = "Unknown"

            # Mid → Up 전이 시 카운트 증가
            if state == "Mid" and current_state == "Up":
                counter += 1

            state = current_state
            prev_elbow_angle = elbow_angle

            # 8. 어깨 궤적 기록 및 시각화
            pts = trajectory[side]
            pts.append((int(SHOULDER[0]), int(SHOULDER[1])))
            if len(pts) > trajectory_len:
                pts.pop(0)

            for i in range(1, len(pts)):
                cv2.line(image, pts[i - 1], pts[i], (0, 255, 255), 2)  # 노란 선

            # 9. 프레임 정보 기록
            frame_info.update({
                "side_used": side,
                "frame_width": width,
                "frame_height": height,
                "elbow_angle": elbow_angle,
                "shoulder_xy": [round(float(SHOULDER[0]), 5), round(float(SHOULDER[1]), 5)],  # 예: [x좌표, y좌표]
                "side_visibility": round(r_vis if r_vis > l_vis else l_vis, 5) # ← 리스트 없이 float 단일 값으로
            })
            log_data.append(frame_info)

            # 10. 랜드마크 시각화
            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=results.pose_landmarks,
                connections=mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_styles.get_default_pose_landmarks_style(),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255), thickness=2)
            )

        # 11. 상태 텍스트 시각화
        cv2.putText(image, f"Frame: {frame_idx}", (30, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        cv2.putText(image, f"Count: {counter}", (30, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)
        if state:
            cv2.putText(image, f"State: {state}", (30, 120),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0,200,255), 2)

        # 12. 영상 출력 저장
        out.write(image)
        frame_idx += 1

# 13. 종료 및 JSON 저장
cap.release()
out.release()
with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(log_data, f, ensure_ascii=False, indent=2)

print(f"분석 완료!\n📁 영상 저장: {OUTPUT_PATH}\n🧾 JSON 저장: {OUTPUT_JSON}")

I0000 00:00:1747730134.862610   21223 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1747730134.925915  382991 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747730134.935309  382995 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


분석 완료!
📁 영상 저장: /Users/laxdin24/Desktop/output_wrong_test1.mov
🧾 JSON 저장: output_wrong_test1.json


### 푸쉬업+골반위치 추적, 정보저장

In [18]:
# 1. 라이브러리 불러오기
import cv2
import mediapipe as mp
import numpy as np
import json

# 2. 각도 계산 함수 정의
def calculate_angle(a, b, c):
    """중심점 b 기준으로 a-b-c 사이 각도 계산"""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

# 3. 기본 설정
VIDEO_PATH = "/Users/laxdin24/Desktop/Input_test1/Nomal_test1.mov"
OUTPUT_PATH = "/Users/laxdin24/Desktop/Output_test1/Nomal_test1.mov"
OUTPUT_JSON = "output_Nomal_test1.json"

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# 4. 영상 입출력 준비
cap = cv2.VideoCapture(VIDEO_PATH)
width, height = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# 5. 변수 초기화
frame_idx = 0
log_data = []
state = "Unknown"
prev_elbow_angle = None
counter = 0

trajectory = {"shoulder": [], "hip": []}
trajectory_len = int(fps * 1)

# 6. MediaPipe Pose 모델 시작
with mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    smooth_landmarks=True,
    min_detection_confidence=0.9,
    min_tracking_confidence=0.5
) as pose:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        frame_info = {"frame": frame_idx}

        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark
            def to_np(idx): return np.array([lm[idx].x * width, lm[idx].y * height])
            def get_x(idx): return lm[idx].x

            # 어깨와 골반 중 더 잘 보이는 방향 선택
            r_vis = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].visibility
            l_vis = lm[mp_pose.PoseLandmark.LEFT_SHOULDER].visibility
            side = 'RIGHT' if r_vis > l_vis else 'LEFT'

            # 선택된 방향 기준 관절 추출
            if side == "RIGHT":
                SHOULDER = to_np(mp_pose.PoseLandmark.RIGHT_SHOULDER)
                HIP      = to_np(mp_pose.PoseLandmark.RIGHT_HIP)
                ELBOW    = to_np(mp_pose.PoseLandmark.RIGHT_ELBOW)
                WRIST    = to_np(mp_pose.PoseLandmark.RIGHT_WRIST)
            else:
                SHOULDER = to_np(mp_pose.PoseLandmark.LEFT_SHOULDER)
                HIP      = to_np(mp_pose.PoseLandmark.LEFT_HIP)
                ELBOW    = to_np(mp_pose.PoseLandmark.LEFT_ELBOW)
                WRIST    = to_np(mp_pose.PoseLandmark.LEFT_WRIST)

            # 팔꿈치 각도 계산
            elbow_angle = calculate_angle(SHOULDER, ELBOW, WRIST)

            # 상태 판단 및 카운트 전이 감지
            if prev_elbow_angle is not None:
                if elbow_angle > 140:
                    current_state = "Up"
                elif 100 < elbow_angle <= 140:
                    current_state = "Mid"
                else:
                    current_state = "Down"
            else:
                current_state = "Unknown"

            if state == "Mid" and current_state == "Up":
                counter += 1

            state = current_state
            prev_elbow_angle = elbow_angle

            # 궤적 기록 (어깨, 골반 따로 저장)
            trajectory["shoulder"].append(tuple(map(int, SHOULDER)))
            trajectory["hip"].append(tuple(map(int, HIP)))

            if len(trajectory["shoulder"]) > trajectory_len:
                trajectory["shoulder"].pop(0)
                trajectory["hip"].pop(0)

            # 어깨 궤적 (노란색), 골반 궤적 (하늘색)
            for i in range(1, len(trajectory["shoulder"])):
                cv2.line(image, trajectory["shoulder"][i-1], trajectory["shoulder"][i], (0, 255, 255), 2)
                cv2.line(image, trajectory["hip"][i-1], trajectory["hip"][i], (255, 255, 0), 2)

            # 로그 기록
            frame_info.update({
                "side_used": side,
                "frame_width": width,
                "frame_height": height,
                "elbow_angle": elbow_angle,
                "shoulder_xy": [round(float(SHOULDER[0]), 5), round(float(SHOULDER[1]), 5)],
                "hip_xy": [round(float(HIP[0]), 5), round(float(HIP[1]), 5)],
                "side_visibility": round(max(r_vis, l_vis), 5)
            })
            log_data.append(frame_info)

            # 랜드마크 시각화
            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=results.pose_landmarks,
                connections=mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_styles.get_default_pose_landmarks_style(),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
            )

        # 텍스트 시각화
        cv2.putText(image, f"Frame: {frame_idx}", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        cv2.putText(image, f"Count: {counter}", (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)
        cv2.putText(image, f"State: {state}", (30, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,200,255), 2)

        # 영상 저장
        out.write(image)
        frame_idx += 1

# 종료 및 JSON 저장
cap.release()
out.release()
with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(log_data, f, ensure_ascii=False, indent=2)

print(f"분석 완료!\n영상 저장: {OUTPUT_PATH}\nJSON 저장: {OUTPUT_JSON}")

I0000 00:00:1747787596.008663   21223 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1747787596.079155 2232729 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747787596.088360 2232729 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


분석 완료!
영상 저장: /Users/laxdin24/Desktop/Output_test1/Nomal_test1.mov
JSON 저장: output_Nomal_test1.json


### 평균이동 보정기법 적용
* 이동평균 필터를 적용해 shoulder_xy와 hip_xy를 부드럽게 만드는 통합 코드 예시는 다음과 같습니다.
한 프레임 기준으로 앞뒤 2프레임씩 총 5프레임 평균을 적용

In [1]:
# ==============================
# 1. 라이브러리 불러오기
# ==============================
import cv2
import mediapipe as mp
import numpy as np
import json

# ==============================
# 2. 각도 계산 함수 정의
# ==============================
def calculate_angle(a, b, c):
    """중심점 b 기준으로 a-b-c 사이 각도 계산 (벡터 내적 이용)"""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

def moving_average(points, k=5):
    """점(x, y)의 리스트를 부드럽게 하는 이동 평균 필터"""
    if len(points) < k:
        return points
    smoothed = []
    for i in range(len(points)):
        start = max(0, i - k//2)
        end = min(len(points), i + k//2 + 1)
        window = points[start:end]
        avg_x = np.mean([p[0] for p in window])
        avg_y = np.mean([p[1] for p in window])
        smoothed.append((avg_x, avg_y))  # 🔹 실수형으로 유지
    return smoothed

# ==============================
# 3. 기본 설정
# ==============================
VIDEO_PATH = "/Users/laxdin24/Desktop/Input_test1/Pushup_test3.mov"
OUTPUT_PATH = "/Users/laxdin24/Desktop/Output_test1/Pushup_test3_smooth.mov"
OUTPUT_JSON = "/Users/laxdin24/Desktop/Output_json/Output_Pushup_test3.json"

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# ==============================
# 4. 영상 입출력 설정
# ==============================
cap = cv2.VideoCapture(VIDEO_PATH)
width, height = int(cap.get(3)), int(cap.get(4))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(OUTPUT_PATH, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# ==============================
# 5. 변수 초기화
# ==============================
frame_idx = 0
log_data = []
state = "Unknown"
prev_elbow_angle = None
counter = 0
trajectory = {"shoulder": [], "hip": []}
trajectory_len = int(fps * 1)

# ==============================
# 6. MediaPipe Pose 실행
# ==============================
with mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    smooth_landmarks=True,
    min_detection_confidence=0.9,
    min_tracking_confidence=0.9
) as pose:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 🔄 BGR → RGB 변환 및 MediaPipe 처리
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        frame_info = {"frame": frame_idx}

        # ==============================
        # 7. 포즈가 인식된 경우
        # ==============================
        if results.pose_landmarks:
            lm = results.pose_landmarks.landmark
            def to_np(idx): return np.array([lm[idx].x * width, lm[idx].y * height])

            # 🔍 좌우 중 더 잘 보이는 쪽 선택
            r_vis = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].visibility
            l_vis = lm[mp_pose.PoseLandmark.LEFT_SHOULDER].visibility
            side = 'RIGHT' if r_vis > l_vis else 'LEFT'

            # 📍 선택된 쪽 관절 추출
            if side == "RIGHT":
                SHOULDER = to_np(mp_pose.PoseLandmark.RIGHT_SHOULDER)
                HIP      = to_np(mp_pose.PoseLandmark.RIGHT_HIP)
                ELBOW    = to_np(mp_pose.PoseLandmark.RIGHT_ELBOW)
                WRIST    = to_np(mp_pose.PoseLandmark.RIGHT_WRIST)
            else:
                SHOULDER = to_np(mp_pose.PoseLandmark.LEFT_SHOULDER)
                HIP      = to_np(mp_pose.PoseLandmark.LEFT_HIP)
                ELBOW    = to_np(mp_pose.PoseLandmark.LEFT_ELBOW)
                WRIST    = to_np(mp_pose.PoseLandmark.LEFT_WRIST)

            # 💪 팔꿈치 각도 계산
            elbow_angle = calculate_angle(SHOULDER, ELBOW, WRIST)

            # 🔁 푸쉬업 상태 판단
            if prev_elbow_angle is not None:
                if elbow_angle > 140:
                    current_state = "Up"
                elif 100 < elbow_angle <= 140:
                    current_state = "Mid"
                else:
                    current_state = "Down"
            else:
                current_state = "Unknown"

            if state == "Mid" and current_state == "Up":
                counter += 1

            state = current_state
            prev_elbow_angle = elbow_angle

            # ==============================
            # 8. 궤적 저장 및 이동평균 적용
            # ==============================
            trajectory["shoulder"].append(SHOULDER)  # 🔹 실수형 유지
            trajectory["hip"].append(HIP)

            if len(trajectory["shoulder"]) > trajectory_len:
                trajectory["shoulder"].pop(0)
                trajectory["hip"].pop(0)

            smoothed_shoulder = moving_average(trajectory["shoulder"])
            smoothed_hip = moving_average(trajectory["hip"])

            # ==============================
            # 9. 궤적 시각화 (cv2.line 은 정수 좌표 필요)
            # ==============================
            for i in range(1, len(smoothed_shoulder)):
                pt1 = tuple(map(int, smoothed_shoulder[i-1]))
                pt2 = tuple(map(int, smoothed_shoulder[i]))
                cv2.line(image, pt1, pt2, (0, 255, 255), 2)  # 노란색: 어깨

                pt1h = tuple(map(int, smoothed_hip[i-1]))
                pt2h = tuple(map(int, smoothed_hip[i]))
                cv2.line(image, pt1h, pt2h, (255, 255, 0), 2)  # 하늘색: 골반

            # ==============================
            # 10. JSON 로그 저장
            # ==============================
            frame_info.update({
                "frame_width": width,
                "frame_height": height,
                "side_used": side,
                "elbow_angle": elbow_angle,
                "shoulder_xy": [round(smoothed_shoulder[-1][0], 5), round(smoothed_shoulder[-1][1], 5)],
                "hip_xy": [round(smoothed_hip[-1][0], 5), round(smoothed_hip[-1][1], 5)],
                "side_visibility": round(max(r_vis, l_vis), 5)
            })
            log_data.append(frame_info)

            # 🖍️ 전체 랜드마크도 시각화
            mp_drawing.draw_landmarks(
                image=image,
                landmark_list=results.pose_landmarks,
                connections=mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_styles.get_default_pose_landmarks_style(),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)
            )

        # ==============================
        # 11. 텍스트 출력 및 저장
        # ==============================
        cv2.putText(image, f"Frame: {frame_idx}", (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
        cv2.putText(image, f"Count: {counter}", (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)
        cv2.putText(image, f"State: {state}", (30, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,200,255), 2)

        out.write(image)
        frame_idx += 1

# ==============================
# 12. 종료 및 파일 저장
# ==============================
cap.release()
out.release()
with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(log_data, f, ensure_ascii=False, indent=2)

print(f"분석 완료!\n📁 영상 저장: {OUTPUT_PATH}\n🧾 JSON 저장: {OUTPUT_JSON}")

# ==============================
# 13. 평가 함수 정의
# ==============================

from sklearn.linear_model import LinearRegression

def normalize_coords(coords, width, height):
    return np.array([[x / width, y / height] for x, y in coords])

def linear_fit_error(points):
    X = np.array(points[:, 0]).reshape(-1, 1)
    y = points[:, 1]
    model = LinearRegression().fit(X, y)
    y_pred = model.predict(X)
    error = np.mean(np.abs(y - y_pred))
    slope = model.coef_[0]
    return model, error, slope

def sync_error(shoulder_pts, hip_pts):
    dx_s = np.diff(shoulder_pts[:, 0])
    dy_s = np.diff(shoulder_pts[:, 1])
    dx_h = np.diff(hip_pts[:, 0])
    dy_h = np.diff(hip_pts[:, 1])
    error_x = np.mean(np.abs(dx_s - dx_h))
    error_y = np.mean(np.abs(dy_s - dy_h))
    return error_x, error_y

def evaluate_pushup_from_log(log_data):
    width = log_data[0]["frame_width"]
    height = log_data[0]["frame_height"]

    shoulder = normalize_coords([f["shoulder_xy"] for f in log_data], width, height)
    hip = normalize_coords([f["hip_xy"] for f in log_data], width, height)

    _, shoulder_err, shoulder_slope = linear_fit_error(shoulder)
    _, hip_err, hip_slope = linear_fit_error(hip)
    sync_x, sync_y = sync_error(shoulder, hip)
    slope_diff = abs(shoulder_slope - hip_slope)

    # 점수 계산
    raw_score = 100
    penalty = (shoulder_err + hip_err) * 120 + (sync_x + sync_y) * 100 + slope_diff * 10
    final_score = max(0, raw_score - penalty)

    return {
        "shoulder_err": round(shoulder_err, 4),
        "hip_err": round(hip_err, 4),
        "sync_x": round(sync_x, 4),
        "sync_y": round(sync_y, 4),
        "slope_diff": round(slope_diff, 4),
        "score": round(final_score, 2)
    }

# ==============================
# 14. 평가 함수 실행
# ==============================

result = evaluate_pushup_from_log(log_data)
print("\n📊 코어 안정성 분석 결과:")
for k, v in result.items():
    print(f"{k}: {v}")

I0000 00:00:1747813050.721872 2960874 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1747813050.773452 2962772 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747813050.781140 2962772 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747813050.792952 2962777 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


분석 완료!
📁 영상 저장: /Users/laxdin24/Desktop/Output_test1/Pushup_test3_smooth.mov
🧾 JSON 저장: /Users/laxdin24/Desktop/Output_json/Output_Pushup_test3.json

📊 코어 안정성 분석 결과:
shoulder_err: 0.028
hip_err: 0.0144
sync_x: 0.0005
sync_y: 0.0011
slope_diff: 0.359
score: 91.15


# 최종

In [20]:
# ==============================
# 1. 라이브러리 불러오기
# ==============================
import cv2
import mediapipe as mp
import numpy as np
import json
from sklearn.linear_model import LinearRegression
from mediapipe.python.solutions.drawing_styles import get_default_pose_landmarks_style


# ==============================
# 2. 각도 계산 함수
# ==============================
def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba, bc = a - b, c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))


# ==============================
# 3. 이동 평균 필터
# ==============================
def moving_average(points, k=5):
    if len(points) < k:
        return points
    smoothed = []
    for i in range(len(points)):
        window = points[max(0, i - k//2): min(len(points), i + k//2 + 1)]
        avg_x = np.mean([p[0] for p in window])
        avg_y = np.mean([p[1] for p in window])
        smoothed.append((avg_x, avg_y))
    return smoothed


# ==============================
# 4. 점수 평가 관련 함수
# ==============================
def normalize_coords(coords, width, height):
    return np.array([[x / width, y / height] for x, y in coords])

def linear_fit_error(points):
    X = np.array(points[:, 0]).reshape(-1, 1)
    y = np.array(points[:, 1])
    model = LinearRegression().fit(X, y)
    y_pred = model.predict(X)
    error = np.mean(np.abs(y - y_pred))
    return model, error

def sync_error(shoulder_pts, hip_pts):
    dx_s, dy_s = np.diff(shoulder_pts[:, 0]), np.diff(shoulder_pts[:, 1])
    dx_h, dy_h = np.diff(hip_pts[:, 0]), np.diff(hip_pts[:, 1])
    return np.mean(np.abs(dx_s - dx_h)), np.mean(np.abs(dy_s - dy_h))


# ==============================
# 5. 피드백 메시지 생성 함수
# ==============================
def give_feedback(shoulder_err, hip_err, sync_x, sync_y):
    feedback = []

    if shoulder_err > 0.05:
        feedback.append("동작이 앞뒤로 움직여요 ")
    else:
        feedback.append("어깨 움직임이 안정적이에요.")

    if hip_err > 0.05:
        feedback.append("골반이 흔들립니다. 허리를 고정해주세요.")
    else:
        feedback.append("골반 움직임이 안정적이에요.")

    if sync_x > 0.02 or sync_y > 0.02:
        feedback.append("어깨와 골반이 따로 움직입니다. 몸통을 하나처럼 유지하세요.")
    else:
        feedback.append("어깨와 골반이 잘 협응되고 있어요.")

    return feedback

def pelvis_angle_feedback(angle_list):
    avg_angle = np.mean(angle_list)
    if avg_angle >= 185:
        return "허리가 굽혀져 있어요. 몸체가 일직선으로 유지되도록 해주세요."
    elif 175 <= avg_angle < 185:
        return "허리 안정적이에요."
    elif avg_angle < 175:
        return "허리가 밑으로 처져있어요. 복부에 힘을 주고 척추를 펴주세요."


# ==============================
# 6. 푸쉬업 평가
# ==============================
def evaluate_pushup_from_log(log_data, counter=0):
    width, height = log_data[0]["frame_width"], log_data[0]["frame_height"]
    shoulder = normalize_coords([f["shoulder_xy"] for f in log_data], width, height)
    hip = normalize_coords([f["hip_xy"] for f in log_data], width, height)

    _, shoulder_err = linear_fit_error(shoulder)
    _, hip_err = linear_fit_error(hip)
    sync_x, sync_y = sync_error(shoulder, hip)

    pelvis_angles = [f["pelvis_angle"] for f in log_data if "pelvis_angle" in f]
    pelvis_fb = pelvis_angle_feedback(pelvis_angles)

    penalty = (shoulder_err + hip_err) * 120 + (sync_x + sync_y) * 100
    score = max(0, 100 - penalty)

    feedback = give_feedback(shoulder_err, hip_err, sync_x, sync_y)
    feedback.append(pelvis_fb)

    return {
        "score": round(score, 2),
        "counter": counter,
        "feedback": feedback
    }


# ==============================
# 7. 영상 분석 및 결과 반환
# ==============================
def analyze_pushup_video(video_path, output_video_path, output_json_path):
    mp_pose = mp.solutions.pose
    cap = cv2.VideoCapture(video_path)
    width, height = int(cap.get(3)), int(cap.get(4))
    fps = cap.get(cv2.CAP_PROP_FPS)

    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
    log_data, frame_idx = [], 0
    state, prev_elbow_angle, counter = "Unknown", None, 0
    trajectory = {"shoulder": [], "hip": []}
    trajectory_len = int(fps)

    with mp_pose.Pose(static_image_mode=False, model_complexity=1,
                      enable_segmentation=False, smooth_landmarks=True,
                      min_detection_confidence=0.9, min_tracking_confidence=0.9) as pose:

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            frame_info = {"frame": frame_idx}

            if results.pose_landmarks:
                lm = results.pose_landmarks.landmark
                def to_np(idx): return np.array([lm[idx].x * width, lm[idx].y * height])
                r_vis = lm[mp_pose.PoseLandmark.RIGHT_SHOULDER].visibility
                l_vis = lm[mp_pose.PoseLandmark.LEFT_SHOULDER].visibility
                side = 'RIGHT' if r_vis > l_vis else 'LEFT'

                if side == "RIGHT":
                    SHOULDER, HIP = to_np(12), to_np(24)
                    ELBOW, WRIST = to_np(14), to_np(16)
                    KNEE = to_np(26)
                else:
                    SHOULDER, HIP = to_np(11), to_np(23)
                    ELBOW, WRIST = to_np(13), to_np(15)
                    KNEE = to_np(25)

                elbow_angle = calculate_angle(SHOULDER, ELBOW, WRIST)
                pelvis_angle = calculate_angle(SHOULDER, HIP, KNEE)

                if prev_elbow_angle is not None:
                    if elbow_angle > 140:
                        current_state = "Up"
                    elif 100 < elbow_angle <= 140:
                        current_state = "Mid"
                    else:
                        current_state = "Down"
                else:
                    current_state = "Unknown"

                if state == "Mid" and current_state == "Up":
                    counter += 1

                state = current_state
                prev_elbow_angle = elbow_angle

                trajectory["shoulder"].append(SHOULDER)
                trajectory["hip"].append(HIP)

                if len(trajectory["shoulder"]) > trajectory_len:
                    trajectory["shoulder"].pop(0)
                    trajectory["hip"].pop(0)

                smoothed_shoulder = moving_average(trajectory["shoulder"])
                smoothed_hip = moving_average(trajectory["hip"])

                frame_info.update({
                    "frame_width": width,
                    "frame_height": height,
                    "side_used": side,
                    "elbow_angle": elbow_angle,
                    "pelvis_angle": round(pelvis_angle, 2),
                    "shoulder_xy": [round(smoothed_shoulder[-1][0], 5), round(smoothed_shoulder[-1][1], 5)],
                    "hip_xy": [round(smoothed_hip[-1][0], 5), round(smoothed_hip[-1][1], 5)],
                    "side_visibility": round(max(r_vis, l_vis), 5)
                })
                log_data.append(frame_info)

                mp.solutions.drawing_utils.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=get_default_pose_landmarks_style(),
                    connection_drawing_spec=mp.solutions.drawing_utils.DrawingSpec(color=(255,255,255), thickness=2)
                )

            out.write(image)
            frame_idx += 1

    cap.release()
    out.release()

    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(log_data, f, ensure_ascii=False, indent=2)

    result = evaluate_pushup_from_log(log_data, counter=counter)

    return {
        "video_path": output_video_path,
        "json_path": output_json_path,
        "score": result["score"],
        "count": result["counter"],
        "feedback": result["feedback"]
    }


video_path = "/Users/laxdin24/Desktop/Input_test1/Nomal_test1.mov"
output_video_path = "/Users/laxdin24/Desktop/Output_test1/Nomal_test1.mov"
output_json_path = "/Users/laxdin24/Desktop/Output_json/Nomal_test1.json"

analyze_pushup_video(video_path, output_video_path, output_json_path)

I0000 00:00:1747886552.165042  109866 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1747886552.231973  389349 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747886552.242364  389349 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


{'video_path': '/Users/laxdin24/Desktop/Output_test1/Nomal_test1.mov',
 'json_path': '/Users/laxdin24/Desktop/Output_json/Nomal_test1.json',
 'score': 91.33,
 'count': 7,
 'feedback': ['어깨 움직임이 안정적이에요.',
  '골반 움직임이 안정적이에요.',
  '어깨와 골반이 잘 협응되고 있어요.',
  '허리 안정적이에요.']}